In [1]:
from sklearn.datasets import load_iris
from sklearn.linear_model import ElasticNet, Lasso, Ridge, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

In [2]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def relu(x):
    return np.maximum(0, x)


def tanh(x):
    return np.tanh(x)


class Relm:
    def __init__(self, k, reg_type, alpha=1.0, activation='tanh', **kwargs):
        self.alpha = alpha
        self.reg_type = reg_type.lower()
        self.k = k
        self.activation = activation.lower()

        if self.reg_type == 'lasso':
            self.model = Lasso(alpha=self.alpha, **kwargs)
        elif self.reg_type == 'ridge':
            self.model = Ridge(alpha=self.alpha, **kwargs)
        elif self.reg_type == 'elasticnet':
            self.model = ElasticNet(alpha=alpha, **kwargs)
        else:
            raise ValueError('{} regularization invalid'.format(self.reg_type))

        if self.activation == 'sigmoid':
            self.activation_func = sigmoid
        elif self.activation == 'tanh':
            self.activation_func = tanh
        elif self.activation == 'relu':
            self.activation_func = relu
        else:
            raise ValueError('{} activation invalid'.format(self.activation))

    def fit(self, X, y):
        G = self._project_features(X)
        self.model.fit(G, y)    

    def _project_features(self, X):
        n_samples, n_features = X.shape
        self.W_ = np.random.randn(n_features, self.k)
        G = self.activation_func(np.dot(X, self.W_))
        return G

    def predict(self, X):
        G = self._project_features(X)
        return self.model.predict(G)

In [3]:
iris = load_iris()
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [4]:
models = [
    ('Lasso', Lasso()),
    ('Ridge', Ridge()),
    ('Elastic', ElasticNet()),
    ('Relm Ridge', Relm(10000, "ridge", activation='sigmoid', alpha=1.0))
]

results = {}

for name, model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    results[name] = mean_squared_error(y_test, y_pred)

In [5]:
results

{'Elastic': 0.17121007617974512,
 'Lasso': 0.3232082593337074,
 'Relm Ridge': 0.4574210025984755,
 'Ridge': 0.06366212731430407}